## Implementation of hash table data structure
We used Python implementation of dictionaries to store information about airports, currency and aircrafts.
This notebook analyses two implementations of hash tables and compares their performances against the Python intergrated data structure 

### Why Hash Tables?

The main idea at the heart of the hash table data structure is that it allows users to assign keys to elements and then use those keys later to look up or remove elements. Its main feature is to save items in a key-indexed table, where the index is a function of the key.

The data structure needed for our purpose require a dynamic set that supports only the operations of <b> INSERT, SEARCH, DELETE</b>.

Although searching for an element in a hash table can take as long as searching for an element in a linked list —$\mathcal{O}(n)$ time in the worst case — in practice, hashing performs extremely well. Under reasonable assumptions, the average time to search for an element in a hash table is $\mathcal{O}(1)$.

A hash table generalizes the simpler notion of an ordinary array. Directly addressing into an ordinary array makes effective use of our ability to examine an arbitrary position in an array in $\mathcal{O}(1)$ time.

We can take advantage of <b>direct addressing</b> when we can afford to allocate an array that has one position for every possible key. 

A hash table typically uses an array of size proportional to the number of keys actually stored. Instead of using the key as an array index directly, the array index is computed from the key.

### Hash Function

Hash Function: method for computing array index from a key.

With direct addressing, an element with key $k$ is stored in slot $k$. With hashing, this element is stored in slot $h(k)$; that is, we use a hash function $h$ to compute the slot from the key $k$. 
We also say that $h(k)$ is the hash value of key $k$. 

The hash function reduces the range of array indices and hence the size of the array. 

There are three main requirements in implementing a <b>good hash function</b> for a given data type: 
- It should be <b>consistent</b>—equal keys must produce the same hash value. 
- It should be <b>efficient</b> to compute. 
- It should <b>uniformly distribute the keys</b>. 

A good approach derives the hash value in a way that we expect to be independent of any patterns that might exist in the data. For example, the <b><i>“division method”</i></b> computes the hash value as the remainder when the key is divided by a specified prime number. 

This method frequently gives good results, assuming that we choose a prime number that is unrelated to any patterns in the distribution of keys. 

Most hash functions assume that the keys are represented by natural numbers. Thus, if the keys are not natural numbers, we find a way to interpret them as natural numbers.

In the division method for creating hash functions, we map a key $k$ into one of $m$ slots (size of the array) by taking the remainder of $k$ divided by $m$. That is, the hash function is:

$$h(k)=k\mod m$$

In our example the keys are not represented with numbers but with strings of character and digits. We will therefore compute the keys as a sequence of ordinal values.

### Collision resolution
The main issue that is related to this data structure is that two keys may hash to the same slot. We call this situation a collision. 


#### Separate Chaining Symbol Table 

A straightforward and general approach to collision resolution is to build, for each of the $M$ array indices, a linked list of the key-value pairs whose keys hash to that index. 
This method is known as separate chaining because items that collide are chained together in separate linked lists. 

The basic idea is to choose $M$ to be sufficiently large that the lists are sufficiently short to enable efficient search through a two-step process: hash to find the list that could contain the key, then sequentially search through that list for the key. 

Since we have $M$ lists and $N$ keys, the average length of the lists is always $N/M$, no matter how the keys are distributed among the lists. 
Typical choice: $M ∼ N/4$ 

In a separate-chaining implementation, our goal is to choose the table size $M$ to be sufficiently small that we do not waste a huge area of contiguous memory with empty chains but sufficiently large that we do not waste time searching through long chains.

#### Linear Probing 

Another approach to implementing hashing is to store $N$ key-value pairs in a hash table of size $M > N$, relying on empty entries in the table to help with collision resolution. 

The simplest open-addressing method is called linear probing: when there is a collision (when we hash to a table index that is already occupied with a key different from the search key), then we just check the next entry in the table (by incrementing the index). 

We hash the key to a table index, check whether the search key matches the key there, and continue (incrementing the index, wrapping back to the beginning of the table if we reach the end) until finding either the search key or an empty table entry. 

### The implementation of has tables will be tested based on the 'airport.csv' that was given as part of the project documentation since it represents the larger amount of data and hence were a performance analysis could be more meaningful

### We are first going to implement linear probing as a collision resolution method.

we use two lists to create a HashTableLinear class that implements the symbol table ADT that uses linear probing. 

One list, called slots, will hold the key items and a parallel list, called data, will hold the data values. When we look up a key, the corresponding position in the data list will hold the associated data value. 

We decide to test this implementation on larger csv file, the airport.csv file, the file contains 5835 entries and, in order to perform linear probing, the size of the table has to be greater than the amount of key-value pairs sotred.

It is important that the size is a prime number so that the collision resolution algorithm can be as efficient as possible. Therefore, we set the the size of the table to be 5839, which is greater than 5835 and the closest prime number. 

In [1]:
import csv 
import unittest
import sys
from pympler import asizeof #tool to measure the memory behavior of Python objects

In [2]:
class HashTableLinear():

    def __init__(self, size):
        ''' this method initialize the hash table with 2 lists of a given size'''
        self.size = size
        self.slots = [None] * self.size
        self.data = [None] * self.size
    
    def show(self):
        '''shows the 2 lists'''
        print(self.slots)
        print(self.data)
    
    def is_full(self):
        '''returns True if table is full'''
        for elem in self.slots:
            if elem == None:
                return False
        return True
        
    def hashfunction(self, key, size):
        '''this function computes the hash value given a key and table size. The function transform 
        into numerical keys that are not integers. Moreover, each position has a different value, this avoids 
        key with same characters in different orders to be given the same hash value. (ABC will be evaluated
        differently than CBA)'''
        if type(key) != int:
            string = ''.join(key)
            sum = 0
            for i, ch in enumerate(string):
                sum +=  ord(ch) * (i+1)
            key = sum

        return key%size
        
    def put(self,key,data):
        '''this function inserts a new key-value pair'''
        #compute hash value
        hashvalue = self.hashfunction(key,len(self.slots))
        #add key value if slot is empty
        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            #replace value is key is already stored
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  
            #check if hashtable is full
            elif self.is_full() == True:
                print ('Error! Hash Table is full')
            else:
                #if current slot is occupied keep searching until an empty is found
                nextslot = hashvalue
                for i in range (len(self.slots)):
                    #keep looking cyclically 
                    nextslot = (nextslot + 1) % len(self.slots)
                    #once empty slot is found, fill it end exit
                    if self.slots[nextslot] == None:
                        self.slots[nextslot]=key
                        self.data[nextslot]=data
                        break
          
    def get(self,key):
        '''this function return the value from a given key, returns False if the key is not in the table'''
        #compute hash value
        startslot = self.hashfunction(key,len(self.slots))
        #set searching conditions
        data = False
        stop = False
        found = False
        #start looking at hash value
        self.position = startslot
        #iterate through the table
        while self.slots[self.position] != None and not found and not stop:
            #if key found extract value
            if self.slots[self.position] == key:
                found = True
                data = self.data[self.position]
            else:
                #else look in the next position
                self.position=(self.position + 1)%len(self.slots)
                #if we are back to the initial hash value means that there is no key=value
                if self.position == startslot:
                       stop = True
        #returns False if nothing was found, else the value
        return data

    def delete(self, key):
        '''this method deletes a key-value pair and rehashes the table. When deleting in linear probing, 
        the entries added after the removed key must be replaced in order to enable search with hashvalue. 
        The algorithm is as follows:
            1-Find and remove the desired element
            2-Go to the next bucket
            3-If the bucket is empty, quit
            4-If the bucket is full, delete the element in that bucket and re-add it to the hash 
                table using the normal means. The item must be removed before re-adding, 
                because it is likely that the item could be added back into its original spot.
            5-Repeat step 2.'''
        #check if there is something to delete
        if self.get(key) == False:
            print('Error! no value for this key')
        else:
            #use the get method position so set key-value to None
            self.slots[self.position] = None
            self.data[self.position] = None
            #rehash the elements at greater indexes 
            i = 1
            while self.position+i < len(self.slots):
                if self.slots[self.position+i] == None:
                    break
                else:
                    temp_key = self.slots[self.position+i]
                    temp_value = self.data[self.position+i]
                    self.slots[self.position+i] = None
                    self.data[self.position+i] = None
                    self.put(temp_key, temp_value)
                i+=1
    

In [3]:
MyHash = HashTableLinear(5)
MyHash.show()

[None, None, None, None, None]
[None, None, None, None, None]


In [4]:
MyHash.put('AEI', 'sadf')
MyHash.put('AEO', '123')
MyHash.put('UUU', '345')
MyHash.put('ZZZ', 'NEW')
MyHash.put('AAA', 'z')

MyHash.show()

['AEO', 'UUU', 'AEI', 'ZZZ', 'AAA']
['123', '345', 'sadf', 'NEW', 'z']


In [5]:
MyHash.get('AAA')

'z'

In [6]:
MyHash.delete('UUU')
MyHash.show()

['AEO', 'ZZZ', 'AEI', 'AAA', None]
['123', 'NEW', 'sadf', 'z', None]


## Testing the implementation correct functionality against Python dictionary

In [7]:
keys = [23, 9, 32, 88, 12]
values = ['a', 'O', 'k', 'P', 'z']

Python_dict = {}
MyHash = HashTableLinear(5)

class test_MyHash(unittest.TestCase):
    
    def test_put_Python(self):
        Python_dict[23]='a'
        Python_dict[9] = 'O'
        self.assertIn(23, Python_dict)
        self.assertIn(9, Python_dict)
    
    def test_put_MyHash(self):
        MyHash.put(23, 'a')
        MyHash.put(9,'O')
        self.assertIn(23, MyHash.slots)
        self.assertIn(9, MyHash.slots)
        self.assertIn('a', MyHash.data)
        self.assertIn('O', MyHash.data)

    def test_get_Python(self):
        Python_dict[23]='a'
        self.assertEqual(Python_dict[23], 'a')
    
    def test_get_MyHash(self):
        MyHash.put(32, 'k')
        MyHash.put(88,'P')
        self.assertEqual(MyHash.get(32), 'k')
        self.assertEqual(MyHash.get(88), 'P')

    def test_delete_Python(self):
        Python_dict[12]='z'
        del Python_dict[12]
        self.assertNotIn(12, Python_dict)
        
    def test_delete_MyHash(self):
        MyHash.put(12,'z')
        self.assertIn(12, MyHash.slots)
        self.assertIn('z', MyHash.data)
        MyHash.delete(12)
        self.assertNotIn(12, MyHash.slots)
        self.assertNotIn('z', MyHash.data) 
        
    
unittest.main(argv=[''], verbosity=2, exit=False)

test_delete_MyHash (__main__.test_MyHash) ... ok
test_delete_Python (__main__.test_MyHash) ... ok
test_get_MyHash (__main__.test_MyHash) ... ok
test_get_Python (__main__.test_MyHash) ... ok
test_put_MyHash (__main__.test_MyHash) ... ok
test_put_Python (__main__.test_MyHash) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


## After confirming that the implementation works correctly, we can test its performances
### testing insertion time

In [8]:
%%time
#insert one element
python_dict_airports = {}
python_dict_airports[123] = 'ABC'

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


In [9]:
%%time
#add only 1 element
myHash_dict_airports = HashTableLinear(10)
myHash_dict_airports.put(123, 'ABC')

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 14.8 µs


In [10]:
%%time
#import all elements from csv
python_dict_airports = {} 

def load_airports_pyDict(file):
    '''loads airports info from file'''
    with open(file, 'r') as csvFile:
        reader = csv.reader(csvFile)
        for row in reader:
            python_dict_airports[row[4]] = (row[2], row[3], row[6], row[7])
    csvFile.close()

load_airports_pyDict('airport.csv')  

CPU times: user 15 ms, sys: 2.59 ms, total: 17.5 ms
Wall time: 16.6 ms


In [11]:
%%time
#import all elements from csv
myHash_dict_airports = HashTableLinear(5839) 

def load_airports_myHash(file):
    '''loads airports info from file'''
    with open(file, 'r') as csvFile:
        reader = csv.reader(csvFile)
        for row in reader:
            myHash_dict_airports.put(row[4], (row[2], row[3], row[6], row[7]))
    csvFile.close()

load_airports_myHash('airport.csv') 

CPU times: user 3.38 s, sys: 11.4 ms, total: 3.39 s
Wall time: 3.4 s


### testing size of the objects

In [12]:
print('size of Python dictionary: ', asizeof.asizeof(python_dict_airports), 'bytes')
print('size of MyHash table: ',  asizeof.asizeof(myHash_dict_airports), 'bytes')

size of Python dictionary:  2525536 bytes
size of MyHash table:  2325240 bytes


### Testing selection time

In [13]:
%%time
python_dict_airports['EUF']

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


('Eufala', 'United States', '31.5708', '-85.0774')

In [14]:
%%time
myHash_dict_airports.get('EUF')

CPU times: user 2.85 ms, sys: 16 µs, total: 2.86 ms
Wall time: 2.94 ms


('Eufala', 'United States', '31.5708', '-85.0774')

### Testing deletion time

In [15]:
%%time
del python_dict_airports['AAA']

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [16]:
%%time
myHash_dict_airports.delete('AAA')

CPU times: user 3.06 s, sys: 10.5 ms, total: 3.07 s
Wall time: 3.08 s


In [17]:
myHash_dict_airports.get('AAA')


False

In [18]:
%%time
del python_dict_airports['CON']

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [19]:
%%time
myHash_dict_airports.delete('CON')

CPU times: user 653 ms, sys: 3.44 ms, total: 656 ms
Wall time: 661 ms


In [20]:
myHash_dict_airports.get('CON')

False

### We now implement chaining as a collision resolution method.

We decide to test this implementation on larger csv file, the airport.csv file, the file contains 5835 entries and, the typical choice is to pick a $table size = number of keys / 4$ which is equal to 1459.

In [21]:
class HashTableChaining():

    def __init__(self, size):
        '''this method initialize a list with a given number of nested lists'''
        self.size = size
        self.table = [[] for _ in range(size)]
    
    def show(self):
        print(self.table)
        
    def hashfunction(self, key,size):
        '''compute hash value as in the linear implementation'''
        if type(key) != int:
            string = ''.join(key)
            sum = 0
            for i, ch in enumerate(string):
                sum +=  ord(ch) * (i+1)
            key = sum

        return key%size
        
    def put(self,key,data):
        '''insert key-value pair'''
        #compute hashvalue
        hashvalue = self.hashfunction(key,len(self.table))
        #find the nested (chained) list
        inner_list = self.table[hashvalue]
        #assume that the key doesn't exist already
        key_exists = False
        #loop through the chain
        for i, kv in enumerate(inner_list):
            k, v = kv
            #check if the key already exists in the table
            if key == k:
                key_exists = True
                break
        #if the key exists, replace the value
        if key_exists:
            inner_list[i] = ((key, data)) 
        #else add the key-value pair to the chained list
        else:
            inner_list.append((key, data)) #add
            

    def get(self,key):
        '''this function return the value from a given key, returns False if the key is not in the table'''
        #compute hashvalue
        hashvalue = self.hashfunction(key,len(self.table))
        #find the nested (chained) list
        inner_list = self.table[hashvalue]
        data = False
        #loop through the chain
        for i, kv in enumerate(inner_list):
            k, v = kv
            if key == k:
                data = v
        return data
        
    def delete(self, key):
        '''this method deletes a key-value pair. The deletion in chaining is simpler as the key-value
        pairs that come after the deleted one are not affected'''
        #compute hashvalue
        hashvalue = self.hashfunction(key,len(self.table))
        #assume that the key doesn't exist
        key_exists = False
        #find the nested (chained) list
        inner_list = self.table[hashvalue]
        #loop through the chain
        for i, kv in enumerate(inner_list):
            k, v = kv
            if key == k:
                key_exists = True
                break
        #if the key exists, delete the elemtn containg key-value pair
        if key_exists:
            del inner_list[i] 
        else:
            print ("Error! no value for this key")


In [22]:
MyHash = HashTableChaining(5)
MyHash.show()

[[], [], [], [], []]


In [23]:
MyHash.put('AEI', 'sadf')
MyHash.put('AEO', '123')
MyHash.put('UUU', '345')
MyHash.put('ZZZ', 'NEW')
MyHash.put('AAA', 'z')

MyHash.show()

[[('AEO', '123'), ('UUU', '345'), ('ZZZ', 'NEW'), ('AAA', 'z')], [], [('AEI', 'sadf')], [], []]


In [24]:
MyHash.get('AAA')

'z'

In [25]:
MyHash.delete('UUU')
MyHash.show()

[[('AEO', '123'), ('ZZZ', 'NEW'), ('AAA', 'z')], [], [('AEI', 'sadf')], [], []]


## Testing the implementation correct functionality against Python dictionary

In [26]:
import csv 
import unittest
keys = [23, 9, 32, 88, 12]
values = ['a', 'O', 'k', 'P', 'z']

Python_dict = {}
MyHash = HashTableChaining(2)

class test_MyHash(unittest.TestCase):
    
    def test_put_Python(self):
        Python_dict[23]='a'
        Python_dict[9] = 'O'
        self.assertIn(23, Python_dict)
        self.assertIn(9, Python_dict)
    
    def test_put_MyHash(self):
        MyHash.put(23, 'a')
        MyHash.put(9,'O')
        self.assertEqual(MyHash.get(23),'a')
        self.assertEqual(MyHash.get(9), 'O')

    def test_get_Python(self):
        Python_dict[23]='a'
        self.assertEqual(Python_dict[23], 'a')
    
    def test_get_MyHash(self):
        MyHash.put(32, 'k')
        MyHash.put(88,'P')
        self.assertEqual(MyHash.get(32), 'k')
        self.assertEqual(MyHash.get(88), 'P')

    def test_delete_Python(self):
        Python_dict[12]='z'
        del Python_dict[12]
        self.assertNotIn(12, Python_dict)
        
    def test_delete_MyHash(self):
        MyHash.put(12,'z')
        MyHash.delete(12)
        self.assertNotIn('z', MyHash.table) 
        
    
unittest.main(argv=[''], verbosity=2, exit=False)

test_delete_MyHash (__main__.test_MyHash) ... ok
test_delete_Python (__main__.test_MyHash) ... ok
test_get_MyHash (__main__.test_MyHash) ... ok
test_get_Python (__main__.test_MyHash) ... ok
test_put_MyHash (__main__.test_MyHash) ... ok
test_put_Python (__main__.test_MyHash) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


### Testing import time

In [27]:
%%time
#insert one element
python_dict_airports = {}
python_dict_airports[123] = 'ABC'

CPU times: user 409 µs, sys: 4 µs, total: 413 µs
Wall time: 419 µs


In [28]:
%%time
#insert one element
myHash_dict_airports = HashTableChaining(10) 
myHash_dict_airports.put(123, 'ABC')

CPU times: user 771 µs, sys: 62 µs, total: 833 µs
Wall time: 839 µs


In [29]:
%%time
#import all elements from csv
python_dict_airports = {} 

def load_airports_pyDict(file):
    '''loads airports info from file'''
    with open(file, 'r') as csvFile:
        reader = csv.reader(csvFile)
        for row in reader:
            python_dict_airports[row[4]] = (row[2], row[3], row[6], row[7])
    csvFile.close()

load_airports_pyDict('airport.csv')

CPU times: user 14.9 ms, sys: 2.69 ms, total: 17.6 ms
Wall time: 15.3 ms


In [30]:
%%time
#import all elements from csv
myHash_dict_airports = HashTableChaining(1459) 

def load_airports_myHash(file):
    '''loads airports info from file'''
    with open(file, 'r') as csvFile:
        reader = csv.reader(csvFile)
        for row in reader:
            myHash_dict_airports.put(row[4], (row[2], row[3], row[6], row[7]))
    csvFile.close()

load_airports_myHash('airport.csv') 

CPU times: user 52.3 ms, sys: 3.49 ms, total: 55.8 ms
Wall time: 54.4 ms


### testing size of the objects

In [31]:
print('size of Python dictionary: ', asizeof.asizeof(python_dict_airports), 'bytes')
print('size of MyHash table: ',  asizeof.asizeof(myHash_dict_airports), 'bytes')

size of Python dictionary:  2525536 bytes
size of MyHash table:  2764352 bytes


### Testing selection time

In [32]:
%%time
python_dict_airports['EUF']

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


('Eufala', 'United States', '31.5708', '-85.0774')

In [33]:
%%time
myHash_dict_airports.get('EUF')

CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 38.9 µs


('Eufala', 'United States', '31.5708', '-85.0774')

### Testing deletion time

In [34]:
%%time
del python_dict_airports['MCO']

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [35]:
%%time
myHash_dict_airports.delete('MCO')

CPU times: user 31 µs, sys: 1 µs, total: 32 µs
Wall time: 36.2 µs


In [36]:
myHash_dict_airports.get('MCO')

False

### Comparing implementations and Python data structure

| Operations     | Python dict   | Linear probing implementation | Chaining implementation |
|----------------|---------------|-------------------------------|-------------------------|
|Insert (1 element)    |  6.91 µs      |    16 µs         |     17.9 µs    |
|Insert (all elements)    | 16.6 ms      |    3.38 s         |     51.5 ms    |
|Search          |  9.06 µs   | 2.95 ms  |  43.2 µs  |
|Delete          | 9.78 µs | 3.06 s| 37 µs |
|Memory Space (bytes)   |  2525536        |   2325240             | 2764352   |


The <b>performance analysis</b> shows that the Python implementation has a lower time of execution for all the main operation of a dictionary.

We can observe that the <b>chaining implementation</b> has worse, but similar, performances. 

On the other hand the <b>clinear probing implementation</b> has much worse perfomances for the operations of insert and delete. This is due to the fact that while adding a new value, if the key hashes to the same index, the algorithm needs to search for an available empty slot one position at the time. This can take a long time especially in a case like ours where we are dealing with thousands of entries. A similar issue affects also the operation of deletion. This because removing an item requires to rehash all the elements with greater indexes in order to make these elements discoverable using the searching algorithm that computes the hashvalue. 

In terms of <b>memory usage</b>, all three implementations have similar values. The linear probing implementation uses a slighly lower amount of memory (2.3Mb) compared to Python's dict (2.5Mb) and the chaining implementation (2.8Mb). This is probably due to the fact that all the data is stored in two separate lists and not in nested objects.

<b>After these evaluations, we decided to use the native Python implementation of dictionaries to store the information contained in the csv files as it increases the speed of the algorithm.</b>